In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

def run[A](code: => A): A = {
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/11/22 20:27:59 INFO SparkContext: Running Spark version 2.4.3
21/11/22 20:27:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/22 20:27:59 INFO SparkContext: Submitted application: dfa7a1f2-7c6b-40b5-beb6-7b851ed29505
21/11/22 20:27:59 INFO SecurityManager: Changing view acls to: oscar
21/11/22 20:27:59 INFO SecurityManager: Changing modify acls to: oscar
21/11/22 20:27:59 INFO SecurityManager: Changing view acls groups to: 
21/11/22 20:27:59 INFO SecurityManager: Changing modify acls groups to: 
21/11/22 20:27:59 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(oscar); groups with view permissions: Set(); users  with modify permissions: Set(oscar); groups with modify permissions: Set()
21/11/22 20:28:00 INFO Utils: Successfully started service 'sparkDrive

21/11/22 20:28:00 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4-sources.jar at spark://netrunner:42025/jars/mainargs_2.12-0.1.4-sources.jar with timestamp 1637609280306
21/11/22 20:28:00 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4.jar at spark://netrunner:42025/jars/mainargs_2.12-0.1.4.jar with timestamp 1637609280306
21/11/22 20:28:00 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1-sources.jar at spark://netrunner:42025/jars/os-lib_2.12-0.7.1-sources.jar with timestamp 1637609280307
21/11/22 20:28:00 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1.jar at spark://netrunner:42025/jars/os-lib_2.12-0.7.1.j

21/11/22 20:28:00 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/alexarchambault/argonaut-shapeless_6.2_2.12/1.2.0-M11/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar at spark://netrunner:42025/jars/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar with timestamp 1637609280311
21/11/22 20:28:00 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.19.v20190610/jetty-server-9.4.19.v20190610.jar at spark://netrunner:42025/jars/jetty-server-9.4.19.v20190610.jar with timestamp 1637609280311
21/11/22 20:28:00 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/io/argonaut/argonaut_2.12/6.2.3/argonaut_2.12-6.2.3.jar at spark://netrunner:42025/jars/argonaut_2.12-6.2.3.jar with timestamp 1637609280311
21/11/22 20:28:00 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/chuusai/shapeless_2.

Spark UI

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                              


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@26d1f49f
import spark.implicits._


defined function run

In [2]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [3]:
val lastRace = Window.partitionBy("year")

val lastRaces = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/races.csv")
    .where(col("year") >= 1990 && col("year") <= 1999)
    .withColumn("round", col("round").cast(IntegerType))
    .withColumn("max", max(col("round")).over(lastRace))
    .where(col("round") === col("max"))
    .select("raceId", "year")

load at cmd2.sc:6

1 / 1

lastRace: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7174dd99
lastRaces: DataFrame = [raceId: string, year: string]

In [7]:
val constructors = Window.partitionBy("constructorId")

val constructorWinners = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/constructor_standings.csv")
    .join(lastRaces, Seq("raceId"), "right")
    .where(col("position") === 1)
    .select("constructorId", "wins", "year")
    .withColumn("totalChampWins", count(col("constructorId")).over(constructors))
    .withColumn("totalRaceWins", sum(col("wins")).over(constructors).cast(IntegerType))
    .drop("wins")  

load at cmd6.sc:6

1 / 1

constructors: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@26f4b9ee
constructorWinners: DataFrame = [constructorId: string, year: string ... 2 more fields]

In [8]:
val constructors = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/constructors.csv")
    .select("constructorId", "name")

load at cmd7.sc:4

1 / 1

constructors: DataFrame = [constructorId: string, name: string]

In [9]:
val mostDominantConstructor = constructorWinners
    .drop("year")
    .dropDuplicates("constructorId")
    .orderBy(col("totalChampWins").desc, col("totalRaceWins").desc)
    .join(constructors, "constructorId")
    .drop("constructorId")
    .show

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

...

show at cmd8.sc:6

1 / 1

show at cmd8.sc:6

3 / 3

+--------------+-------------+--------+
|totalChampWins|totalRaceWins|    name|
+--------------+-------------+--------+
|             5|           47|Williams|
|             3|           23| McLaren|
|             1|           11|Benetton|
|             1|            6| Ferrari|
+--------------+-------------+--------+

